In [ ]:
onnx_model_path = "./model/model.onnx"

ExecutionProvider="QNNExecutionProvider"
if ExecutionProvider == "OpenVINOExecutionProvider":
    onnx_model_path = "./model/ov_model_st_quant.onnx"

In [ ]:
def _fix_winrt_runtime():
    """This function removes the msvcp140.dll from the winrt-runtime package.
    So it does not cause issues with other libraries.
    """
    from importlib import metadata
    from pathlib import Path
    site_packages_path = Path(str(metadata.distribution('winrt-runtime').locate_file('')))
    dll_path = site_packages_path / 'winrt' / 'msvcp140.dll'
    if dll_path.exists():
        dll_path.unlink()
            
def _get_ep_paths() -> dict[str, str]:
    from winui3.microsoft.windows.applicationmodel.dynamicdependency.bootstrap import (
        InitializeOptions,
        initialize
    )
    import winui3.microsoft.windows.ai.machinelearning as winml
    eps = {}
    with initialize(options = InitializeOptions.ON_NO_MATCH_SHOW_UI):
        catalog = winml.ExecutionProviderCatalog.get_default()
        providers = catalog.find_all_providers()
        for provider in providers:
            provider.ensure_ready_async().get()
            eps[provider.name] = provider.library_path
            # DO NOT call provider.try_register in python. That will register to the native env.
    return eps

def _regsiter_executino_providers_to_onnxruntime():
    import onnxruntime as ort

    paths = _get_ep_paths()
    for item in paths.items():
        print(f"----register ort ep---- {item[0]} {item[1]}")
        ort.register_execution_provider_library(item[0], item[1])

_fix_winrt_runtime()
_regsiter_executino_providers_to_onnxruntime()

In [ ]:
import onnxruntime as ort
from transformers import BertTokenizer
import numpy as np

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the ONNX model
def add_ep_for_device(session_options, ep_name, device_type, ep_options=None):
    ep_devices = ort.get_ep_devices()
    for ep_device in ep_devices:
        if ep_device.ep_name == ep_name and ep_device.device.type == device_type:
            print(f"Adding {ep_name} for {device_type}")
            session_options.add_provider_for_devices([ep_device], {} if ep_options is None else ep_options)
            break


session_options = ort.SessionOptions()

add_ep_for_device(session_options, ExecutionProvider, ort.OrtHardwareDeviceType.NPU)

session = ort.InferenceSession(
    onnx_model_path, # a model wirh QNN EPContext nodes
    sess_options=session_options,
)

def areParaphrases(sentence1, sentence2):
    # Tokenize the input sentences
    inputs = tokenizer(sentence1, sentence2, return_tensors='np', padding='max_length', truncation=True, max_length=128)

    # Convert inputs to numpy arrays
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']

    # Run inference
    outputs = session.run(None, {
        'input_ids': input_ids.astype(np.int64),
        'attention_mask': attention_mask.astype(np.int64),
        'token_type_ids': token_type_ids.astype(np.int64)
    })

    # Get the prediction
    logits = outputs[0]
    predicted_label = np.argmax(logits, axis=1)
    return predicted_label == 1

In [ ]:
# Prepare the input data
sentence1 = "The company Hugging Face is based in New York City."
sentence2 = "Hugging Face's headquarters are situated in NYC."
sentence3 = "New York City is big."

# Interpret the result
if areParaphrases(sentence1, sentence2):
    print("Expected: The sentences are paraphrases.")
else:
    print("Not expected: The sentences are not paraphrases.")

if areParaphrases(sentence1, sentence3):
    print("Not expected: The sentences are paraphrases.")
else:
    print("Expected: The sentences are not paraphrases.")